In [1]:
from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.palettes import Viridis, Category10, Category20
from bokeh.io import export_svg
from tqdm import tqdm
output_notebook()

Loading BokehJS ...

In [2]:
from metamod.tasks import TaskSwitch, AffineCorrelatedGaussian, CompositionOfTasks, SemanticTask
from metamod.trainers import two_layer_training, two_layer_engage_training
from metamod.networks import LinearNet, LinearTaskEngNet
from metamod.control import TaskSwitchLinearNetEq, TaskSwitchLinearNetControl
from metamod.utils import plot_lines

Loading BokehJS ...

## Baseline linear network

In [3]:
run_name = "composition_of_tasks"
results_path = "../results"
results_dict = {}

In [4]:
dataset_params = {"batch_size": 32,
                  "h_levels": 3}

dataset = SemanticTask(**dataset_params)

In [5]:
model_params = {"learning_rate": 5e-3,
                "hidden_dim": 10,
                "intrinsic_noise": 0.0,
                "reg_coef": 0.0,
                "input_dim": dataset.input_dim,
                "output_dim": dataset.output_dim,
                "W1_0": None,
                "W2_0": None}

model = LinearNet(**model_params)

In [6]:
n_steps = 5000
save_weights_every = 20

iters, baseline_loss, weights_iter, weights = two_layer_training(model=model, dataset=dataset, n_steps=n_steps, save_weights_every=save_weights_every)

results_dict["iters"] = iters
results_dict["Loss_t_sim"] = baseline_loss
results_dict["weights_sim"] = weights
results_dict["weights_iters_sim"] = weights_iter

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:04<00:00, 1217.87it/s]


In [7]:
losses = (baseline_loss, )
colors = (Category10[10][0], )
legends = ("Baseline linear network",)
alphas = (1, )

s = plot_lines(iters, losses, legends, alphas, colors)
show(s)

## Composition of tasks test, same task but different code..

In [8]:
composition_dataset_params = {"dataset_classes": (SemanticTask,), 
                              "dataset_list_params": (dataset_params,)}

In [9]:
composition_dataset = CompositionOfTasks(**composition_dataset_params)

In [10]:
comp_model_params = {"learning_rate": 5e-3,
                "hidden_dim": 10,
                "intrinsic_noise": 0.0,
                "reg_coef": 0.0,
                "input_dim": composition_dataset.input_dim,
                "output_dim": composition_dataset.output_dim,
                "W1_0": weights[0][0, ...],
                "W2_0": weights[1][0, ...],
                "task_output_index": composition_dataset.task_output_index}

engage_coefficients = np.ones((n_steps, 1))  # (t, phis)

comp_model = LinearTaskEngNet(**comp_model_params)

In [11]:
iters, comp_loss, weights_iter, weights = two_layer_engage_training(model=comp_model, 
                                                                    dataset=composition_dataset, 
                                                                    n_steps=n_steps, 
                                                                    save_weights_every=save_weights_every,
                                                                    engagement_coefficients=engage_coefficients)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:04<00:00, 1069.06it/s]


In [12]:
losses = (baseline_loss, comp_loss)
colors = (Category10[10][0], Category10[10][1])
legends = ("Baseline linear network", "compositional task")
alphas = (0.5, 0.5)

s = plot_lines(iters, losses, legends, alphas, colors)
show(s)

## Trying different tasks

In [13]:
dataset_params1 = dataset_params.copy()
dataset_params2 = {"batch_size": 32,
                  "h_levels": 4}
composition_dataset_params = {"dataset_classes": (SemanticTask, SemanticTask), 
                              "dataset_list_params": (dataset_params1, dataset_params2)}

In [14]:
composition_dataset = CompositionOfTasks(**composition_dataset_params)

In [15]:
comp_model_params = {"learning_rate": 5e-3,
                "hidden_dim": 10,
                "intrinsic_noise": 0.0,
                "reg_coef": 0.0,
                "input_dim": composition_dataset.input_dim,
                "output_dim": composition_dataset.output_dim,
                "W1_0": None,
                "W2_0": None,
                "task_output_index": composition_dataset.task_output_index}

engage_coefficients = np.ones((n_steps, len(composition_dataset.datasets)))  # (t, phis)
engage_coefficients[:, 1] = np.linspace(0, 1, num=engage_coefficients.shape[0])
engage_coefficients[:, 0] = np.linspace(1, 0, num=engage_coefficients.shape[0])

comp_model = LinearTaskEngNet(**comp_model_params)

In [16]:
iters, comp_loss, weights_iter, weights = two_layer_engage_training(model=comp_model, 
                                                                    dataset=composition_dataset, 
                                                                    n_steps=n_steps, 
                                                                    save_weights_every=save_weights_every,
                                                                    engagement_coefficients=engage_coefficients)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:06<00:00, 761.96it/s]


In [17]:
losses = (baseline_loss, comp_loss)
colors = (Category10[10][0], Category10[10][1])
legends = ("Baseline linear network", "compositional task")
alphas = (0.5, 0.5)

s = plot_lines(iters, losses, legends, alphas, colors)
show(s)

In [18]:
composition_dataset.task_output_index, composition_dataset.task_input_index

([(0, 7), (7, 22)], [(0, 4), (4, 12)])